# Library functions

Here are the functions that initialise the road randomly and update it. We import numpy simply to get the random number generator - all the work is done using standard Python lists.

In [12]:
# %load trafficlib.py
import numpy as np

def initroad(road, density, seedval):

#  Here we expect a road without halos    

    np.random.seed(seedval)

    n = len(road)

    ncar = 0

    for i in range(0, n):
        
        rng = np.random.random()

        if rng < density:
            road[i] = 1
        else:
            road[i] = 0

        ncar = ncar + road[i]

    return ncar


def updateroad(newroad, oldroad):

    n = len(oldroad)-2

    nmove = 0

    for i in range(1, n+1):

        if oldroad[i] == 1:
            if oldroad[i+1] == 1:
                newroad[i] = 1
            else:
                newroad[i] = 0
                nmove = nmove + 1
        else:
            if oldroad[i-1] == 1:
                newroad[i] = 1
            else:
                newroad[i] = 0

#
#  The version below will be easier to express as array operations
#  as it has fewer "if" statements.
#
#        if oldroad[i] == 0:
#            newroad[i] = oldroad[i-1]
#        else:
#            newroad[i] = oldroad[i+1]
#
#        if newroad[i] != oldroad[i]:
#            nmove = nmove + 1
#
#    nmove = nmove/2
#

    return nmove


def updatebcs(road):

    n = len(road)-2

    road[0]   = road[n]
    road[n+1] = road[1]


import time

def gettime():

    return time.time()


# Main program

Here is the main program. Note that with these default parameters it will take a **long** time (several minutes) to run, but we will use this length of road for the numpy examples so it is worth running it at least once. The slightly strange value for `ncell` is because, when we run in parallel, we will require that the `ncell` is an exact multiple of the number of processes we use (just for simplicity). The number of processes is often a power of two, so this value lets us run on process counts from 1, 2, 4, ..., 2048, 4096 on ARCHER2. On your laptop I would not recommend using more than 8 processes!

The number of iterations auto-adjusts to keep the runtime relatively constant. However, if you want to experiment with the performance using shorter roads then I would suggest reducing this, e.g. set `maxiter = 102400000//ncell`.

 * Run the program and note the execution speed in MCOPs.
 * Reduce the value of maxiter as suggested above and check you get roughly the same speed.
 * Now try using a road which is a factor of ten smaller - does this affect the speed? What about a very short road of 1024?


In [ ]:
# %load traffic.py
#!/usr/bin/env python

import sys
import time

from trafficlib import initroad, updatebcs, updateroad, gettime

def main(argv):

    # Simulation parameters
    seedval = 5743
    ncell = 10240000
    maxiter = 102400000//ncell
    printfreq = maxiter//10

    tmproad = [0 for col in range(ncell)]
    newroad = [0 for col in range(ncell+2)]
    oldroad = [0 for col in range(ncell+2)]

    density = 0.52

    print(f"Length of road is {ncell}")
    print(f"Number of iterations is {maxiter}")
    print(f"Target density of cars is {density}")

    # Initialise road accordingly using random number generator
    print(f"Initialising ...")

    ncars = initroad(tmproad, density, seedval)

    print(f"Actual Density of cars is {float(ncars)/float(ncell)}\n")

    oldroad[1:ncell+1] = tmproad
    
    tstart = gettime()

    for iter in range(1, maxiter+1):

        updatebcs(oldroad)

        nmove = updateroad(newroad, oldroad)

        # Copy new to old array
        for i in range(1, ncell+1):
            oldroad[i] = newroad[i]

        if iter % printfreq == 0:

          print(f"At iteration {iter} average velocity is {float(nmove)/float(ncars):.6f}")

    tstop = gettime()

    print(f"\nFinished\n")
    print(f"Time taken was {tstop-tstart:.2f} seconds")
    print(f"Update rate was {1.0e-6*ncell*maxiter/(tstop-tstart):.2f} MCOPs")

if __name__ == "__main__":
    main(sys.argv[1:])


Length of road is 10240000
Number of iterations is 10
Target density of cars is 0.52
Initialising ...
Actual Density of cars is 0.520013671875

At iteration 1 average velocity is 0.480019
At iteration 2 average velocity is 0.599724
At iteration 3 average velocity is 0.659508
At iteration 4 average velocity is 0.696773
At iteration 5 average velocity is 0.722954
At iteration 6 average velocity is 0.742499
At iteration 7 average velocity is 0.757813
At iteration 8 average velocity is 0.770349
